In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime, timedelta

//anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [3]:
def SMAMeanReversionBacktest(ticker, sma, threshold, safety=False, safety_threshold=0.25, start_date='2000-01-01',end_date='2020-12-31'):
    """
    Perform a backtest of the SMA mean reversion strategy
    
    Inputs
        ticker (str) - stock ticker
        sma (int) - rolling average window size, in days
        threshold (float) - mean reversion significance threshold
        start_date (str)
        end_date (str)
    Outputs
        data (dataframe) - contains strategy returns and statistics data
    """
    #Get yahooFinance historical data
    yfObj = yf.Ticker(ticker)
    data = yfObj.history(start=start_date, end=end_date)
    #Calculate SMA and extension at the end of each day
    data['SMA'] = data['Close'].rolling(sma).mean()
    data['extension'] = (data['Close'] - data['SMA']) / data['SMA']
    
    #Check the extension at the end of each day and adjust the position accordingly
    if safety==True:
        data['position'] = np.nan
        data['position'] = np.where((data['extension']<-threshold) & (data['extension']>-safety_threshold), 1, data['position'])
        data['position'] = np.where(np.abs(data['extension'])<0.01, 0, data['position'])
        data['position'] = data['position'].ffill().fillna(0)
    else:
        data['position'] = np.nan
        data['position'] = np.where(data['extension']<-threshold, 1, data['position'])
        data['position'] = np.where(np.abs(data['extension'])<0.01, 0, data['position'])
        data['position'] = data['position'].ffill().fillna(0)
    
    #Calculate returns and statistics
    data['returns'] = data['Close'] / data['Close'].shift(1)
    data['log_returns'] = np.log(data['returns'])
    data['strat_returns'] = data['position'].shift(1) * data['returns']
    data['strat_log_returns'] = data['position'].shift(1) * data['log_returns']
    data['cum_returns'] = np.exp(data['log_returns'].cumsum())
    data['strat_cum_returns'] = np.exp(data['strat_log_returns'].cumsum())
    data['peak'] = data['cum_returns'].cummax()
    data['strat_peak'] = data['strat_cum_returns'].cummax()
    
    return data.dropna()

In [4]:
def stratBacktest(ticker, strat, sma, threshold, safety=False, safety_threshold=0.25, start_date='2000-01-01',end_date='2020-12-31'):
    """
    Perform a backtest of a specified strategy
    
    Inputs
        ticker (str) - stock ticker
        sma (int) - rolling average window size, in days
        threshold (float) - mean reversion significance threshold
        start_date (str)
        end_date (str)
    Outputs
        data (dataframe) - contains strategy returns and statistics data
    """
    #Get yahooFinance historical data
    yfObj = yf.Ticker(ticker)
    data = yfObj.history(start=start_date, end=end_date)
    #Calculate SMA and extension at the end of each day
    data['SMA'] = data['Close'].rolling(sma).mean()
    data['extension'] = (data['Close'] - data['SMA']) / data['SMA']
    data['SMA_short_long'] = data['Close'].rolling(short_term_sma).mean() / data['Close'].rolling(long_term_sma).mean()
    
    #Check the strategy criteria at the end of each day and adjust the position accordingly
    data = strat_decision(data, strat, threshold, safety_threshold, short_long_threshold, safety)
    
    #Calculate returns and statistics
    data['returns'] = data['Close'] / data['Close'].shift(1)
    data['log_returns'] = np.log(data['returns'])
    data['strat_returns'] = data['position'].shift(1) * data['returns']
    data['strat_log_returns'] = data['position'].shift(1) * data['log_returns']
    data['cum_returns'] = np.exp(data['log_returns'].cumsum())
    data['strat_cum_returns'] = np.exp(data['strat_log_returns'].cumsum())
    data['peak'] = data['cum_returns'].cummax()
    data['strat_peak'] = data['strat_cum_returns'].cummax()
    
    return data.dropna()

In [5]:
def getStratStats(data, risk_free_rate=0.02):
    sma_strat, buy_hold_strat = {}, {}
    
    #Total Returns
    sma_strat['tot_returns'] = np.exp(data['strat_log_returns'].sum()) - 1
    buy_hold_strat['tot_returns'] = np.exp(data['log_returns'].sum()) - 1
    
    #Mean Annual Returns
    sma_strat['annual_returns'] = np.exp(data['strat_log_returns'].mean()*252) - 1
    buy_hold_strat['annual_returns'] = np.exp(data['log_returns'].mean()*252) - 1
    
    #Annual Volatility
    sma_strat['annual_volatility'] = data['strat_log_returns'].std() * np.sqrt(252)
    buy_hold_strat['annual_volatility'] = data['log_returns'].std() * np.sqrt(252)
    
    #Sharpe Ratio
    sma_strat['sharpe_ratio'] = (sma_strat['annual_returns'] - risk_free_rate) / sma_strat['annual_volatility']
    buy_hold_strat['sharpe_ratio'] = (buy_hold_strat['annual_returns'] - risk_free_rate) / buy_hold_strat['annual_volatility']
    
    #Max Drawdown
    _strat_dd = data['strat_peak'] - data['strat_cum_returns']
    _buy_hold_dd = data['peak'] - data['cum_returns']
    sma_strat['max_drawdown'] = _strat_dd.max()
    buy_hold_strat['max_drawdown'] = _buy_hold_dd.max()
    
    #Max Drawdown Duration
    strat_dd = _strat_dd[_strat_dd==0]
    strat_dd_diff = strat_dd.index[1:] - strat_dd.index[:-1]
    strat_dd_days = strat_dd_diff.map(lambda x: x.days).values
    strat_dd_days = np.hstack([strat_dd_days, (_strat_dd.index[-1] - strat_dd.index[-1]).days])
    
    buy_hold_dd = _buy_hold_dd[_buy_hold_dd==0]
    buy_hold_dd_diff = buy_hold_dd.index[1:] - buy_hold_dd.index[:-1]
    buy_hold_dd_days = buy_hold_dd_diff.map(lambda x: x.days).values
    buy_hold_dd_days = np.hstack([buy_hold_dd_days, (_buy_hold_dd.index[-1] - buy_hold_dd.index[-1]).days])
    
    sma_strat['max_drawdown_duration'] = strat_dd_days.max()
    buy_hold_strat['max_drawdown_duration'] = buy_hold_dd_days.max()
    
    stats_dict = {'strat_stats': sma_strat, 'base_stats': buy_hold_strat}

    return stats_dict

In [6]:
#Apply backtest to a ticker
#Set parameters
ticker = 'ETH-CAD'
start_date = '2018-12-31'
end_date = '2020-12-31'
strat='SMA_MR'
SMA = 50
threshold = 0.1
safety_threshold = 0.15
#yfObj = yf.Ticker(ticker)
#Apply backtest
data_basic = SMAMeanReversionBacktest(ticker, SMA, threshold, safety=False,start_date=start_date,end_date=end_date)
data_safety = SMAMeanReversionBacktest(ticker, SMA, threshold, safety=True, safety_threshold=safety_threshold,start_date=start_date,end_date=end_date)
data_safety_ensemble = stratBacktest(ticker, strat, SMA, threshold, safety=True, safety_threshold=safety_threshold,start_date=start_date,end_date=end_date)
#Evaluate backtest statistics for each strategy
basic_stats_dict = getStratStats(data_basic)
df_basic_stats = pd.DataFrame(basic_stats_dict).round(3)
safe_stats_dict = getStratStats(data_safety)
df_safe_stats = pd.DataFrame(safe_stats_dict).round(3)
safe_ensemble_stats_dict = getStratStats(data_safety_ensemble)
df_safe_ensemble_stats = pd.DataFrame(safe_ensemble_stats_dict).round(3)

NameError: name 'short_term_sma' is not defined

In [1]:
fig, ax = plt.subplots(figsize=(12, 5))
ax.plot(data_basic['strat_cum_returns'], label='Mean Reversion Strategy')
ax.plot(data_safety['strat_cum_returns'], label='Mean Reversion Strategy with Safety')
ax.plot(data_safety_ensemble['strat_cum_returns'], label='Mean Reversion Strategy with Safety and Ensemble')
ax.plot(data_safety['cum_returns'], label=f'{ticker}')
ax.set_xlabel('Date')
ax.set_ylabel('Returns (%)')
ax.set_title('Cumulative Returns for Mean Reversion and Buy and Hold Strategies')

ax.legend()
plt.show()

NameError: name 'plt' is not defined

In [5]:
print('Basic Strategy Statistics \n')
print(df_basic_stats)
print('\n Safety Threshold Strategy Statistics \n')
print(df_safe_stats)

Basic Strategy Statistics 

                       strat_stats  base_stats
tot_returns                  0.721      -0.470
annual_returns               0.037      -0.041
annual_volatility            0.542       0.687
sharpe_ratio                 0.031      -0.089
max_drawdown                 1.467       3.170
max_drawdown_duration     2790.000    5150.000

 Safety Threshold Strategy Statistics 

                       strat_stats  base_stats
tot_returns                  3.193      -0.470
annual_returns               0.100      -0.041
annual_volatility            0.482       0.687
sharpe_ratio                 0.166      -0.089
max_drawdown                 1.074       3.170
max_drawdown_duration     1280.000    5150.000


In [53]:
def SMAMeanReversion_strat(ticker, sma, threshold, current_position, safety=False, safety_threshold=0.25, short_term_sma=30, long_term_sma=90, short_long_threshold=0.05):
    """
    Evaluate a stock using two versions of SMA mean reversion strategy and make a buy/hold/sell decision
    
    Inputs
        ticker (str) - stock ticker
        sma (int) - rolling average window size, in days
        threshold (float) - mean reversion significance threshold, >0
        current_position (int) - our current position for this stock 
            1 means we own the stock, 0 means we do not
        safety (bool) - if True, activate safety latch
        safety_threshold (float) - safety latch threshold, >0
        short_term_sma (int) - window size for short sma
        long_term_sma (int) - window size for long sma
        short_long_threshold (float) - short_long ratio significance threshold
        
        Must have safety_threshold > threshold
    Outputs
        decision (dataframe) - contains stock ticker, current extension, and strategy decision
    """
    strat = 'SMA_MR'
    
    #end_date = '2021-07-24'
    end_date = datetime.today().strftime('%Y-%m-%d')
    start_date = (datetime.today() - timedelta(days=2*long_term_sma)).strftime('%Y-%m-%d')
    
    #Get yahooFinance historical data
    yfObj = yf.Ticker(ticker)
    data = yfObj.history(start=start_date, end=end_date)
    #Calculate SMA and extension at the end of each day
    data['SMA'] = data['Close'].rolling(sma).mean()
    data['extension'] = (data['Close'] - data['SMA']) / data['SMA']
    data['SMA_short_long'] = data['Close'].rolling(short_term_sma).mean() / data['Close'].rolling(long_term_sma).mean()
    
    #Check the strategy criteria at the end of each day and adjust the position accordingly
    data = strat_decision(data, strat, threshold, safety_threshold, short_long_threshold, safety)

    #Can create a combined criteria that requires both the extension and the short_long ratio to agree in order to make a move
    if (data['extension'][-1:].values<-threshold) & (data['SMA_short_long'][-1:].values<1-short_long_threshold) & (current_position == 0):
        movement = 'Buy'
    else:
        movement = 'Hold'
    #Could make this an OR statement to conservatively sell if either signal is triggered
    if (data['extension'][-1:].values>-0.01) & (data['SMA_short_long'][-1:].values>1) & (current_position == 1):
        movement = 'Sell'
    else:
        if movement != 'Buy':
            movement = 'Hold'
    
      
    #Get the extension & decision for this stock today and store it in a df
    decision_dict = {}
    decision_dict['ticker'] = ticker
    decision_dict['extension'] = data['extension'][-1:]
    decision_dict['extension_position'] = data['extension_position'][-1:]
    decision_dict['short_long ratio'] = data['SMA_short_long'][-1:]
    decision_dict['short_long_position'] = data['short_long_position'][-1:]
    decision_dict['movement'] = movement
    decision_dict['position'] = data['position'][-1:]
    decision = pd.DataFrame(decision_dict).round(3)
    
    return decision, data

In [52]:
def strat_decision(data, strat, threshold, safety_threshold, short_long_threshold, safety):
    """
    Take a stock data dictionary and a strategy with settings, calculate the strategy positions and add them to the
    data dictionary. Return the updated data dictionary.
    """
    if strat=='SMA_MR':
        if safety==True:
            data['extension_position'] = np.nan
            data['extension_position'] = np.where((data['extension']<-threshold) & (data['extension']>-safety_threshold), 1, data['extension_position'])
            data['extension_position'] = np.where(data['extension']>-0.01, 0, data['extension_position'])
            data['extension_position'] = data['extension_position'].ffill().fillna(0)

            data['short_long_position'] = np.nan
            data['short_long_position'] = np.where(data['SMA_short_long']<1-short_long_threshold, 1, data['short_long_position']) #adjusted threshold to 0.95 to leave a hold band around 1
            data['short_long_position'] = np.where(data['SMA_short_long']>1, 0, data['short_long_position'])
            data['short_long_position'] = data['short_long_position'].ffill().fillna(0)

            #Set the position to 1 if both signal positions are 1. Else, set the position to 0.
            data['position'] = np.nan
            data['position'] = data['extension_position']*data['short_long_position']
        else:
            data['extension_position'] = np.nan
            #If extension is below our threshold, buy. Else, hold
            data['extension_position'] = np.where(data['extension']<-threshold, 1, data['extension_position'])
            #If we the extension is within 0.01 of a neutral value, sell. Else, hold
            data['extension_position'] = np.where(data['extension']>-0.01, 0, data['extension_position'])
            data['extension_position'] = data['extension_position'].ffill().fillna(0)

            data['short_long_position'] = np.nan
            data['short_long_position'] = np.where(data['SMA_short_long']<1-short_long_threshold, 1, data['short_long_position'])
            data['short_long_position'] = np.where(data['SMA_short_long']>1, 0, data['short_long_position'])
            data['short_long_position'] = data['short_long_position'].ffill().fillna(0)

            #Set the position to 1 if both signal positions are 1. Else, set the position to 0.
            data['position'] = np.nan
            data['position'] = data['extension_position']*data['short_long_position']
    
    return data

In [51]:
ticker = 'BNB-CAD'
threshold = 0.1
current_position = 0
safety = False
safety_threshold = 0.25
short_term_sma=30 
long_term_sma=90
short_long_threshold=0.05
decision, data = SMAMeanReversion_strat(ticker, 50, threshold, current_position,safety=safety, safety_threshold=safety_threshold, short_term_sma=short_term_sma, long_term_sma=long_term_sma, short_long_threshold=short_long_threshold)
decision
#print(data)

,ticker,extension,extension_position,short_long ratio,short_long_position,movement,position
Date,,,,,,,
2021-09-09,BNB-CAD,0.079,0.0,1.262,0.0,Hold,0.0


Write a function to take a list of tickers and a strategy function and evaluate each of the tickers with that strategy, returning a list of buy/hold/sell moves for a given day

In [8]:
def applyPortfolioStrat(strat_func, tickers, current_positions, SMA, threshold, safety=False, safety_threshold=0.25, short_term_sma=30, long_term_sma=90, short_long_threshold=0.05):
    """
    Take a strategy function and apply it to a list of tickers and current positions.
    Return the decisions in a dataframe.
    
    Should eventually package all of the strategy settings into a strat_settings array variable to make this 
    easier to change, more readable, and robust enough to use different strat_func's with non-identical inputs. 
    Will pack the settings before using applyPortfolioStrat and then unpack inside of strat_func.
    
    applyPortfolioStrat(strat_func, strat_settings, tickers, current_positions)
    SMAMeanReversion_strat(ticker, strat_settings, current_position)
    """
    decisions, _ = strat_func(tickers[0], SMA, threshold, current_positions[0], safety=safety, safety_threshold=safety_threshold, short_term_sma=short_term_sma, long_term_sma=long_term_sma, short_long_threshold=short_long_threshold)
    if len(tickers)>1:
        for i in range(1,len(tickers)):
            decision, _ = strat_func(tickers[i], SMA, threshold, current_positions[i], safety=safety, safety_threshold=safety_threshold, short_term_sma=short_term_sma, long_term_sma=long_term_sma, short_long_threshold=short_long_threshold)
            decisions = decisions.append(decision)
    return decisions

In [9]:
#List of stocks we want to consider in our portfolio
tickers = ['ETH-CAD','BTC-CAD','ADA-CAD','BNB-CAD','SNP','GOOGL','AAPL']
current_positions = [0]*len(tickers)

#Set strategy params
SMA = 50
threshold = 0.1
safety = False
safety_threshold = 0.25
short_term_sma=30 
long_term_sma=90
short_long_threshold=0.05
strat_settings = [SMA, threshold, safety, safety_threshold, short_term_sma, long_term_sma, short_long_threshold]

#Apply strategy to portfolio
decisions = applyPortfolioStrat(SMAMeanReversion_strat, tickers, current_positions, SMA, threshold, safety=safety, safety_threshold=safety_threshold, short_term_sma=short_term_sma, long_term_sma=long_term_sma, short_long_threshold=short_long_threshold)
decisions

,ticker,extension,extension_position,short_long ratio,short_long_position,movement,position
Date,,,,,,,
2021-09-09,ETH-CAD,0.166,0.0,1.290,0.0,Hold,0.0
2021-09-09,BTC-CAD,0.058,0.0,1.208,0.0,Hold,0.0
2021-09-09,ADA-CAD,0.256,0.0,1.473,0.0,Hold,0.0
2021-09-09,BNB-CAD,0.081,0.0,1.262,0.0,Hold,0.0
2021-09-08,SNP,0.065,0.0,0.945,1.0,Hold,0.0
2021-09-08,GOOGL,0.071,0.0,1.094,0.0,Hold,0.0
2021-09-08,AAPL,0.056,0.0,1.076,0.0,Hold,0.0


Plot the performance of our strategy vs buy and hold for a given stock. Do the same thing for a portfolio containing a list of stocks.

In [58]:
#def plotBacktest(ticker, sma, threshold, safety=False, safety_threshold=0.25, short_term_sma=30, long_term_sma=90, short_long_threshold=0.05):
    
 #   end_date = datetime.today().strftime('%Y-%m-%d')
 #   start_date = (datetime.today() - timedelta(weeks=2*52)).strftime('%Y-%m-%d')
    
    
    
    

Weight the size of our position by the magnitude of under-extension instead of unit buy/sell. 
If extension gets large enough, buy more. If extension gets closer to the cutoff at -0.01, sell some.

In [10]:
end_date = datetime.today().strftime('%Y-%m-%d')
start_date = (datetime.today() - timedelta(weeks=2*52)).strftime('%Y-%m-%d')
print(end_date)
print(start_date)

2021-09-09
2019-09-12


In [11]:
msft = yf.Ticker("SNP")

# get stock info
#msft.info

# get historical market data
hist = msft.history(period="2y")

# show actions (dividends, splits)
msft.actions

# show analysts recommendations
#msft.recommendations

# show next event (earnings, etc)
#msft.calendar

#plt.plot(hist['Close'])

,Dividends,Stock Splits
Date,,
2020-06-01,2.690,0.0
2020-10-14,1.023,0.0
2021-06-08,1.984,0.0


Debugging portfolioBacktest error "IndexError: index -1 is out of bounds for axis 0 with size 0" when using full list of tickers (fractional or crypto) ['TSLA','AAPL','TD','AMZN','SHOP','MSFT','RY','BNS','NVDA','CNR','BMO','FTS','ARKK','ABNB','NFLX','COIN','FB','BCE','VOO','DOL','SQ','KO','GOOGL','WMT']

In [111]:
def strat_decision(data, strat, threshold, safety_threshold, short_long_threshold, safety):
    """
    Take a stock data dictionary and a strategy with settings, calculate the strategy positions and add them to the
    data dictionary. Return the updated data dictionary.
    """
    if strat=='SMA_MR':
        if safety==True:
            data['extension_position'] = np.nan
            data['extension_position'] = np.where((data['extension']<-threshold) & (data['extension']>-safety_threshold), 1, data['extension_position'])
            data['extension_position'] = np.where(data['extension']>-0.01, 0, data['extension_position'])
            data['extension_position'] = data['extension_position'].ffill().fillna(0)

            data['short_long_position'] = np.nan
            data['short_long_position'] = np.where(data['SMA_short_long']<1-short_long_threshold, 1, data['short_long_position']) #adjusted threshold to 0.95 to leave a hold band around 1
            data['short_long_position'] = np.where(data['SMA_short_long']>1, 0, data['short_long_position'])
            data['short_long_position'] = data['short_long_position'].ffill().fillna(0)

            #Set the position to 1 if both signal positions are 1. Else, set the position to 0.
            data['position'] = np.nan
            data['position'] = data['extension_position']*data['short_long_position']
        else:
            data['extension_position'] = np.nan
            #If extension is below our threshold, buy. Else, hold
            data['extension_position'] = np.where(data['extension']<-threshold, 1, data['extension_position'])
            #If we the extension is within 0.01 of a neutral value, sell. Else, hold
            data['extension_position'] = np.where(data['extension']>-0.01, 0, data['extension_position'])
            data['extension_position'] = data['extension_position'].ffill().fillna(0)

            data['short_long_position'] = np.nan
            data['short_long_position'] = np.where(data['SMA_short_long']<1-short_long_threshold, 1, data['short_long_position'])
            data['short_long_position'] = np.where(data['SMA_short_long']>1, 0, data['short_long_position'])
            data['short_long_position'] = data['short_long_position'].ffill().fillna(0)

            #Set the position to 1 if both signal positions are 1. Else, set the position to 0.
            data['position'] = np.nan
            data['position'] = data['extension_position']*data['short_long_position']
    
    return data

def stratBacktest(ticker, strat, sma, threshold, safety=False, safety_threshold=0.25, short_term_sma=30, long_term_sma=90, short_long_threshold=0.05, start_date='2000-01-01',end_date='2020-12-31'):
    """
    Perform a backtest of a specified strategy for a given ticker
    
    Inputs
        ticker (str) - stock ticker
        sma (int) - rolling average window size, in days
        threshold (float) - mean reversion significance threshold
        start_date (str)
        end_date (str)
    Outputs
        data (dataframe) - contains strategy returns and statistics data
    """
    #Get yahooFinance historical data
    yfObj = yf.Ticker(ticker)
    data = yfObj.history(start=start_date, end=end_date)
    #Calculate SMA and extension at the end of each day
    data['SMA'] = data['Close'].rolling(sma).mean()
    data['extension'] = (data['Close'] - data['SMA']) / data['SMA']
    data['SMA_short_long'] = data['Close'].rolling(short_term_sma).mean() / data['Close'].rolling(long_term_sma).mean()
    
    #Check the strategy criteria at the end of each day and adjust the position accordingly
    data = strat_decision(data, strat, threshold, safety_threshold, short_long_threshold, safety)
    
    #Calculate returns and statistics
    data['returns'] = data['Close'] / data['Close'].shift(1)
    data['log_returns'] = np.log(data['returns'])
    data['strat_returns'] = data['position'].shift(1) * data['returns']
    data['strat_log_returns'] = data['position'].shift(1) * data['log_returns']
    data['cum_returns'] = np.exp(data['log_returns'].cumsum())
    data['strat_cum_returns'] = np.exp(data['strat_log_returns'].cumsum())
    data['peak'] = data['cum_returns'].cummax()
    data['strat_peak'] = data['strat_cum_returns'].cummax()
    
    return data.dropna()

def sum_metrics(tickers, portfolio_returns, portfolio_stats):
    
    #Get the returns and stats from the first ticker
    portfolio_sum_returns = portfolio_returns[tickers[0]]
    portfolio_sum_stats = portfolio_stats[tickers[0]]
    #For each remaining ticker, add the returns and stats to the portfolio total sum
    if len(tickers)>1:
        for i in range(1,len(tickers)):
            portfolio_sum_returns = portfolio_sum_returns + portfolio_returns[tickers[i]]
            portfolio_sum_stats = portfolio_sum_stats + portfolio_stats[tickers[i]]
    #Re-normalize the portfolio total sum
    portfolio_sum_returns = portfolio_sum_returns/len(tickers)
    #Not sure exactly how to normalize the stats
    #portfolio_sum_stats = portfolio_sum_stats/len(tickers)
    
    return portfolio_sum_returns, portfolio_sum_stats

def getStratStats(data, verbose, risk_free_rate=0.02):
    sma_strat, buy_hold_strat = {}, {}
    
    #Total Returns
    sma_strat['tot_returns'] = np.exp(data['strat_log_returns'].sum()) - 1
    buy_hold_strat['tot_returns'] = np.exp(data['log_returns'].sum()) - 1
    
    #Mean Annual Returns
    sma_strat['annual_returns'] = np.exp(data['strat_log_returns'].mean()*252) - 1
    buy_hold_strat['annual_returns'] = np.exp(data['log_returns'].mean()*252) - 1
    
    #Annual Volatility
    sma_strat['annual_volatility'] = data['strat_log_returns'].std() * np.sqrt(252)
    buy_hold_strat['annual_volatility'] = data['log_returns'].std() * np.sqrt(252)
    
    #Sharpe Ratio
    #sma_strat['sharpe_ratio'] = (sma_strat['annual_returns'] - risk_free_rate) / sma_strat['annual_volatility']
    #buy_hold_strat['sharpe_ratio'] = (buy_hold_strat['annual_returns'] - risk_free_rate) / buy_hold_strat['annual_volatility']
    
    #Max Drawdown
    _strat_dd = data['strat_peak'] - data['strat_cum_returns']
    _buy_hold_dd = data['peak'] - data['cum_returns']
    sma_strat['max_drawdown'] = _strat_dd.max()
    buy_hold_strat['max_drawdown'] = _buy_hold_dd.max()
    
    #Max Drawdown Duration
    strat_dd = _strat_dd[_strat_dd==0]
    strat_dd_diff = strat_dd.index[1:] - strat_dd.index[:-1]
    strat_dd_days = strat_dd_diff.map(lambda x: x.days).values
    strat_dd_days = np.hstack([strat_dd_days, (_strat_dd.index[-1] - strat_dd.index[-1]).days])
        
    sma_strat['max_drawdown_duration'] = strat_dd_days.max()

    try:
        buy_hold_dd = _buy_hold_dd[_buy_hold_dd==0]
        buy_hold_dd_diff = buy_hold_dd.index[1:] - buy_hold_dd.index[:-1]
        buy_hold_dd_days = buy_hold_dd_diff.map(lambda x: x.days).values
        buy_hold_dd_days = np.hstack([buy_hold_dd_days, (_buy_hold_dd.index[-1] - buy_hold_dd.index[-1]).days])
        #Calculate max drawdown duration as largest difference between reaching a peak value and then coming back to that value after falling
        buy_hold_strat['max_drawdown_duration'] = buy_hold_dd_days.max()
    except IndexError:
        if verbose:
            print('IndexError occured due to no drawdown occurences')
        buy_hold_strat['max_drawdown_duration'] = 0.0
    
    
    stats_dict = {'strat_stats': sma_strat, 'base_stats': buy_hold_strat}

    return stats_dict

def portfolioBacktest(args):
        """
        Apply the backtest function to every ticker in a list. Create a dictionary containing
        the running cumulative returns data of each ticker, and a dictionary containing the 
        statistical performance of each ticker.
        
        Need to handle the errors caused by:
            - stock didn't exist at the beginning of given time frame
            - divide by zero error
            - index -1 out of range for array of size 0
            (Not sure why these second two errors occur)
        """
        [tickers, strat, SMA, threshold, safety_threshold, short_term_sma, long_term_sma, short_long_threshold, start_date, end_date, verbose] = args
        
        #Store the returns and stats of each ticker in the portfolio
        portfolio_returns = {}
        portfolio_stats = {}
        for i in range(0,len(tickers)):
            if verbose:
                print('Backtesting', tickers[i])
            #Get the backtest returns and stats for ticker i
            data_df = stratBacktest(tickers[i], strat, SMA, threshold, safety=True, safety_threshold=safety_threshold, short_term_sma=short_term_sma, long_term_sma=long_term_sma, short_long_threshold=short_long_threshold, start_date=start_date,end_date=end_date)
            stats_df = pd.DataFrame(getStratStats(data_df, verbose)).round(3)
            returns_df = data_df[['cum_returns','strat_cum_returns']]

            #Store the returns and stats of the ticker
            portfolio_returns[tickers[i]] = returns_df
            portfolio_stats[tickers[i]] = stats_df
        
        #Sum the returns and stats across the portfolio
        portfolio_sum_returns, portfolio_sum_stats = sum_metrics(tickers, portfolio_returns, portfolio_stats)
        
        return portfolio_sum_returns, portfolio_sum_stats

In [105]:
tickers = ['TSLA','AAPL','TD','AMZN','SHOP','MSFT','RY','BNS','NVDA','CNR','BMO','FTS','ARKK','ABNB','NFLX','COIN','FB','BCE','VOO','DOL','SQ','KO','GOOGL','WMT']
#Working:'TSLA','AAPL','AMZN','SHOP','MSFT','NVDA','FTS','ARKK','ABNB','NFLX','RY','FB','BCE','VOO','DOL','SQ','KO','GOOGL','WMT'
#IndexError:'COIN','TD','BNS','CNR','BMO'
verbose = True

start_date = '2021-05-01'
end_date = '2021-10-01'
strat='SMA_MR'
SMA = 30
threshold = 0.1
safety_threshold = 0.2
short_term_sma=10
long_term_sma=30
short_long_threshold=0.05

#Store the returns and stats of each ticker in the portfolio
portfolio_returns = {}
portfolio_stats = {}
for i in range(0,len(tickers)):
    if verbose:
        print('Backtesting', tickers[i])
    #Get the backtest returns and stats for ticker i
    data_df = stratBacktest(tickers[i], strat, SMA, threshold, safety=True, safety_threshold=safety_threshold, short_term_sma=short_term_sma, long_term_sma=long_term_sma, short_long_threshold=short_long_threshold, start_date=start_date,end_date=end_date)
    stats = getStratStats(data_df, verbose)
    stats_df = pd.DataFrame(stats).round(3)
    returns_df = data_df[['cum_returns','strat_cum_returns']]

    #Store the returns and stats of the ticker
    portfolio_returns[tickers[i]] = returns_df
    portfolio_stats[tickers[i]] = stats_df

#Sum the returns and stats across the portfolio
portfolio_sum_returns, portfolio_sum_stats = sum_metrics(tickers, portfolio_returns, portfolio_stats)

In [95]:
tickers = ['TSLA']
#Working:'TSLA','AAPL','AMZN','SHOP','MSFT','NVDA','FTS','ARKK','ABNB','NFLX','RY','FB','BCE','VOO','DOL','SQ','KO','GOOGL','WMT'
#IndexError:'COIN','TD','BNS','CNR','BMO'
verbose = True

start_date = '2021-05-01'
end_date = '2021-10-01'
strat='SMA_MR'
SMA = 30
threshold = 0.1
safety_threshold = 0.2
short_term_sma=10
long_term_sma=30
short_long_threshold=0.05

#portfolioBacktest()
#Store the returns and stats of each ticker in the portfolio
portfolio_returns = {}
portfolio_stats = {}

i=0
if verbose:
    print('Backtesting', tickers[i])
    
#Get the backtest returns and stats for ticker i
data_df = stratBacktest(tickers[i], strat, SMA, threshold, safety=True, safety_threshold=safety_threshold, short_term_sma=short_term_sma, long_term_sma=long_term_sma, short_long_threshold=short_long_threshold, start_date=start_date,end_date=end_date)

#getStratStats()
data = data_df
sma_strat, buy_hold_strat = {}, {}  
#Total Returns
sma_strat['tot_returns'] = np.exp(data['strat_log_returns'].sum()) - 1
buy_hold_strat['tot_returns'] = np.exp(data['log_returns'].sum()) - 1

#Mean Annual Returns
sma_strat['annual_returns'] = np.exp(data['strat_log_returns'].mean()*252) - 1
buy_hold_strat['annual_returns'] = np.exp(data['log_returns'].mean()*252) - 1

#Annual Volatility
sma_strat['annual_volatility'] = data['strat_log_returns'].std() * np.sqrt(252)
buy_hold_strat['annual_volatility'] = data['log_returns'].std() * np.sqrt(252)

#Max Drawdown
_strat_dd = data['strat_peak'] - data['strat_cum_returns']
_buy_hold_dd = data['peak'] - data['cum_returns']
#print(_buy_hold_dd)
sma_strat['max_drawdown'] = _strat_dd.max()
buy_hold_strat['max_drawdown'] = _buy_hold_dd.max()

#Max Drawdown Duration
strat_dd = _strat_dd[_strat_dd==0]
strat_dd_diff = strat_dd.index[1:] - strat_dd.index[:-1]
strat_dd_days = strat_dd_diff.map(lambda x: x.days).values
strat_dd_days = np.hstack([strat_dd_days, (_strat_dd.index[-1] - strat_dd.index[-1]).days])

sma_strat['max_drawdown_duration'] = strat_dd_days.max()

try:
    buy_hold_dd = _buy_hold_dd[_buy_hold_dd==0]
    buy_hold_dd_diff = buy_hold_dd.index[1:] - buy_hold_dd.index[:-1]
    buy_hold_dd_days = buy_hold_dd_diff.map(lambda x: x.days).values
    buy_hold_dd_days = np.hstack([buy_hold_dd_days, (_buy_hold_dd.index[-1] - buy_hold_dd.index[-1]).days])
    #Calculate max drawdown duration as largest difference between reaching a peak value and then coming back to that value after falling
    buy_hold_strat['max_drawdown_duration'] = buy_hold_dd_days.max()
except IndexError:
    print('IndexError occured due to buy_hold_dd = _buy_hold_dd[_buy_hold_dd==0]. \nDummy value of zero used for max_drawdown_duration.')
    buy_hold_strat['max_drawdown_duration'] = 0.0

stats_dict = {'strat_stats': sma_strat, 'base_stats': buy_hold_strat}
#end getStratStats()

stats_df = pd.DataFrame(stats_dict).round(3)
returns_df = data_df[['cum_returns','strat_cum_returns']]

#Store the returns and stats of the ticker
portfolio_returns[tickers[i]] = returns_df
portfolio_stats[tickers[i]] = stats_df

#Sum the returns and stats across the portfolio
portfolio_sum_returns, portfolio_sum_stats = sum_metrics(tickers, portfolio_returns, portfolio_stats)
#end portfolioBacktest()


Backtesting TSLA


In [96]:
#buy_hold_dd_days = np.hstack([buy_hold_dd_days, (_buy_hold_dd.index[-1] - buy_hold_dd.index[-1]).days])
print(min(_buy_hold_dd))

0.0


In [112]:
#tickers = ['TSLA','AAPL','TD','AMZN','SHOP','MSFT','RY','BNS','NVDA','CNR','BMO','FTS','ARKK','ABNB','NFLX','COIN','FB','BCE','VOO','DOL','SQ','KO','GOOGL','WMT']
tickers = ['TSLA','AAPL']
verbose = True

start_date = '2015-05-01'
end_date = '2021-10-01'
strat='SMA_MR'
SMA = 30
threshold = 0.1
safety_threshold = 0.2
short_term_sma=10
long_term_sma=30
short_long_threshold=0.05

args = [tickers, strat, SMA, threshold, safety_threshold, short_term_sma, long_term_sma, short_long_threshold, start_date, end_date, verbose]

portfolio_returns, portfolio_stats = portfolioBacktest(args)

Backtesting TSLA
Backtesting AAPL
